# Implementation of HED in Pytorch
---


In [ ]:
# import torch libraries
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# import utility functions
from model import HED
from trainer import Trainer
from dataproc import BSDSDataset

In [ ]:
# fix random seed
rng = np.random.RandomState(37148)

# GPU ID
gpuID = 1

### Preprocess and Setup Data

In [ ]:
# load the images dataset
dataRoot = 'data/HED-BSDS/'
testPath = dataRoot+'test.lst'
trainPath = dataRoot+'train_pair.lst'

rootDir = dataRoot
frame = pd.read_csv(trainPath, dtype=str, delimiter=' ')

# check some sample images
idx = 5

# input and target images
inputName = os.path.join(rootDir, frame.iloc[idx, 0])
targetName = os.path.join(rootDir, frame.iloc[idx, 1])        

# process the images
inputImage = io.imread(inputName)
inputImage = inputImage.astype(np.float32)
inputImage = inputImage[:,:,::-1]

targetImage = io.imread(targetName, as_grey=True)
targetImage = targetImage > 0
targetImage = targetImage.astype(np.float32)

plt.figure()
plt.imshow(inputImage)
plt.show()

plt.figure()
plt.imshow(targetImage,cmap=cm.gray)
plt.show()

inputImage -= np.array((104.00698793,116.66876762,122.67891434))
inputImage /= 255*np.array((0.225, 0.224, 0.229))
print(inputImage.mean(axis=(0,1)), inputImage.std(axis=(0,1)))

inputImage = inputImage.transpose((2,0,1))
inputImage = np.expand_dims(inputImage, axis=0)
inputImage = torch.from_numpy(np.flip(inputImage, axis=0).copy())

targetImage = np.expand_dims(targetImage, axis=0)        
targetImage = np.expand_dims(targetImage, axis=0)        
targetImage = torch.from_numpy(targetImage)

### Train the Network

In [ ]:
# batch size
nBatch = 1

# load the images dataset
dataRoot = 'data/HED-BSDS/'
valPath = dataRoot+'val_pair.lst'
trainPath = dataRoot+'train_pair.lst'

# create data loaders from dataset
valDataset = BSDSDataset(valPath, dataRoot)
trainDataset = BSDSDataset(trainPath, dataRoot)

valDataloader = DataLoader(valDataset, batch_size=nBatch)
trainDataloader = DataLoader(trainDataset, batch_size=nBatch)

In [ ]:
# initialize the network
net = HED()
net.cuda(gpuID)

# define the optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-5, momentum=0.9, weight_decay=0.0002)

# initialize trainer class
trainer = Trainer(net, optimizer, trainDataloader, valDataloader, cuda=True, gpuID=gpuID)

# train the network
trainer.train()